In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from imblearn.under_sampling import NearMiss
from keras.models import Sequential
from keras.layers import Dense
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pandas_profiling import ProfileReport

C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.e

In [2]:
data=pd.read_csv("train_ctrUa4K.csv")
for column in ('Gender','Married','Dependents','Self_Employed'):
    data[column].fillna(data[column].mode()[0],inplace=True)
for column in ('LoanAmount','Loan_Amount_Term','Credit_History'):
    data[column].fillna(data[column].mean(),inplace=True)
for variable in ('Gender','Married','Dependents','Education','Self_Employed','Property_Area'):
    data[variable].fillna("Missing",inplace=True)
    dummies=pd.get_dummies(data[variable],prefix=variable)
    data=pd.concat([data,dummies],axis=1)
    data.drop([variable],axis=1,inplace=True)
data['Loan_Status']=data.Loan_Status.map({'Y':0,'N':1})
Y=data['Loan_Status']
data.drop(['Loan_Status'],axis=1,inplace=True)
X=data[data.iloc[:,1:23].columns]
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=100,test_size=0.2)
scaler=StandardScaler()
scaled_X_train=scaler.fit_transform(X_train)
scaled_X_test=scaler.transform(X_test)

In [3]:
def decisiontree_param(X, y, nfolds):
    min_sample_leaf_val=[1,2,3,4,5,6,7,8,9,10]
    criterion_val=['entropy','gini']
    max_depth=[1,2,3,4,5,6,7,8,9,10]
    param_grid = {'max_depth':max_depth,'criterion':criterion_val,'min_samples_leaf':min_sample_leaf_val}
    DT_model=DecisionTreeClassifier()
    grid_search_DT = GridSearchCV(estimator=DT_model, param_grid=param_grid, cv=nfolds)
    grid_search_DT.fit(X,y)
    return grid_search_DT.best_params_
decisiontree_param(scaled_X_train,Y_train,3)

{'criterion': 'entropy', 'max_depth': 1, 'min_samples_leaf': 1}

In [4]:
DT_model=DecisionTreeClassifier(criterion = "entropy",max_depth=1, min_samples_leaf=1)
DT_model.fit(scaled_X_train,Y_train)
DT_pred=DT_model.predict(scaled_X_test)
print("Recall for decision tree model:",metrics.recall_score(Y_test,DT_pred))
print("Precision for decision tre model:",metrics.precision_score(Y_test,DT_pred))
print("Accuracy for decision tre model:",metrics.accuracy_score(Y_test,DT_pred))
print("F-score for decision tre model:",metrics.f1_score(Y_test,DT_pred))
print("Log-loss for decision tre model:",metrics.log_loss(Y_test,DT_pred))

Recall for decision tree model: 0.375
Precision for decision tre model: 0.8571428571428571
Accuracy for decision tre model: 0.7317073170731707
F-score for decision tre model: 0.5217391304347825
Log-loss for decision tre model: 9.266520486376773
